In [ ]:
import google.generativeai as genai
from crewai_tools import JSONSearchTool
import openai
import os
from llm import llm
from json_writer import JSONWriterTool

openai.api_key = os.getenv("OPENAI_API_KEY")
genai.configure(api_key=os.getenv("google_ai_api_key"))

In [2]:
financial_data_tool = JSONSearchTool(json_path='userForm.json',
                                     config={
                                         "embedder": {
                                             "provider": "google",
                                             "config": {
                                                 "model": "models/embedding-001",
                                                 "task_type": "retrieval_document"
                                             },
                                         }
                                     })

json_writer_tool = JSONWriterTool(
    json_path='updatedUserForm.json',
    config={
        "writer": {
            "provider": "local",
            "config": {
                "write_mode": "overwrite",
                "format": "json",
                "indent": 4
            }
        }
    }
)


Agents

In [6]:
from crewai import Agent

In [7]:
# agent to read json file

read_json_agent = Agent(
    role='Read JSON Input Agent',
    goal='Read the JSON file for information on demographics, financials, risk tolerance, and investment preferences.',
    # verbose=True,
    memory=True,
    backstory=(
        "Expert in extracting information from json input"
    ),
    tools=[financial_data_tool],
    allow_delegation=True,
    llm=llm
)

In [8]:
# agent to assess financial goals

financial_goals_assement_agent = Agent(
    role='Get information on financial goals Agent',
    goal="""Read the JSON file for information on financial goals and provide an updated analysis on how it affects risk tolerance. 
            Use this information to reassess the user's risk tolerance based on: Current income, investments, and debt levels,Investment goals 
            and preferences, Existing tolerance levels and target values.""",
    # verbose=True,
    memory=True,
    backstory=(
        "Expert in extracting information related to financial goals from the data"
    ),
    # tools=[financial_data_tool],
    allow_delegation=True,
    llm=llm
)


In [9]:
# Create a investment strategy agent

investment_strategy_agent = Agent(
    role='Investment strategy Agent',
    goal="""Analyze financial data to assess investment strategy and provide an updated analysis on how it affects risk tolerance.
            Use this information to reassess the user's risk tolerance based on: Current income, investments, and debt levels, Investment goals
            and preferences, Existing tolerance levels and target values.""",
    # verbose=True,
    memory=True,
    backstory=(
        "Expert in extracting information related to investment strategy from the data"
    ),
    # tools=[financial_data_tool],
    allow_delegation=True,
    llm=llm
)



In [10]:
# Create a portfolio structure agent

portfolio_structure_agent = Agent(
    role='Portfolio structure Agent',
    goal="""Analyze financial data to assess portfolio structure and provide an updated analysis on how it affects risk tolerance.'
            Use this information to reassess the user's risk tolerance based on: Current income, investments, and debt levels, Investment goals
            and preferences, Existing tolerance levels and target values.""",
    # verbose=True,
    memory=True,
    backstory=(
        "Expert in extracting information related to portfolio structure from the data"
    ),
    # tools=[financial_data_tool],
    allow_delegation=True,
    llm=llm
)


In [11]:
# agents to compile information from all other agents
final_compilation_agent = Agent(
    role='Final Compilation Agent',
    goal='Compile findings from all agents and update the risk tolerance values in a new JSON file.',
    # verbose=True,
    memory=True,
    backstory=(
        "Expert in compiling data from various sources and updating the risk tolerance values based on analysis."
    ),
    # tools=[financial_data_tool, json_writer_tool],
    allow_delegation=False,
    llm=llm
)

Tasks

In [12]:
from crewai import Task

In [13]:
# Risk Assessment Task
read_json_task = Task(
    description=(
        "Analyze the input file and provide the information on demographics, financials, risk tolerance, and investment preferences."
    ),
    expected_output='The information on demographics, financials, risk tolerance, and investment preferences.',
    tools=[financial_data_tool],
    agent=read_json_agent,
    llm=llm
)

financial_goals_assessment_task = Task(
    description=(
        "Extract the financial goals information from the input file and analyze the financial goals data."
    ),
    expected_output='The information and analysis of the financial goals.',
    # tools=[financial_data_tool],
    agent=financial_goals_assement_agent,
    llm=llm
)

investment_strategy_task = Task(
    description=(
        "Analyze the financial data to assess the client’s risk tolerance and provide an investment strategy based on the provided information."
    ),
    expected_output='The analysis of risk tolerance and a proposed investment strategy.',
    # tools=[financial_data_tool],
    agent=investment_strategy_agent,
    llm=llm
)
portfolio_structure_task = Task(
    description=(
        "Analyze the financial data to design a portfolio structure that matches the client’s risk tolerance and financial goals."
    ),
    expected_output='A detailed portfolio structure based on the client’s financial data.',
    # tools=[financial_data_tool],
    agent=portfolio_structure_agent,
    llm=llm
)

final_compilation_task = Task(
    description=(
        "Gather findings from the read_json_agent, financial_goals_assessment_agent, "
        "investment_strategy_agent, and portfolio_structure_agent. Compile the data, update the user's risk tolerance values "
        "to reflect changes in their financial situation or risk parameters, and write a new JSON file."
    ),
    expected_output="A new JSON file with updated risk tolerance values based on the client's current financial situation and risk preferences.",
    # tools=[financial_data_tool,json_writer_tool],
    agent=final_compilation_agent,
    llm=llm
)

Running the agents

In [14]:
from crewai import Crew, Process

In [ ]:
crew = Crew(
    # agents=[read_json_agent],
    agents=[read_json_agent, financial_goals_assement_agent,
            investment_strategy_agent, portfolio_structure_agent, final_compilation_agent],
    manager_llm=llm,
    planning_llm=llm,
    # tasks=[read_json_task],
    tasks=[read_json_task, financial_goals_assessment_task,
           investment_strategy_task, portfolio_structure_task, final_compilation_task],
    process=Process.sequential,
    verbose=True,
    memory=True,
    embedder={
        "provider": "google",
        "config": {
            "model": 'models/embedding-001',
            "task_type": "retrieval_document",
            "title": "Embeddings for Embedchain"
        }
    },
    cache=True,
    max_rpm=100,
    share_crew=True,
    output_log_file='crew_execution.log'
)


In [ ]:
# Start the task execution process
result = crew.kickoff()

In [ ]:
print(result.raw)

In [24]:
import re


def extract_json(text):
    # Use regular expression to extract the JSON part
    json_match = re.search(r'\{.*\}', text, re.DOTALL)
    if json_match:
        return json_match.group(0)
    else:
        return None

In [ ]:
json_data = extract_json(result.raw)

if json_data: print(json_data) 
else: print("No JSON found")

In [27]:
final_data = extract_json(result.raw)

with open("data.json", "w") as file:
    file.write(str(final_data))

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True